In [1]:
from typing import Text, Generator, Tuple, List, Optional, Dict, Set
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import json
import random
import re
import base64
import ast
import copy
import uuid
import imgkit
from tabulate import tabulate
sns.set_theme()

config = imgkit.config(wkhtmltoimage='../wkhtmltopdf/bin/wkhtmltoimage.exe')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:.2f}'.format

# 1. AWS Functions

In [218]:
import base64
import json
import boto3
from PIL import Image
import io
import uuid

S3_CLIENT = boto3.client('s3')
BUCKET_NAME = 'bucket_name'
BASE_HTML_URL = "base_url"
PLACEHOLDER_URL = "placeholder"

def upload_html_to_s3(html_bytes_io, bucket_name, object_name):
    S3_CLIENT.upload_fileobj(
        html_bytes_io,
        bucket_name,
        object_name,
        ExtraArgs={'ContentType': 'text/html'}
    )

def generate_link_and_upload(generated_html, row_name):
    curr_uuid = str(uuid.uuid4())
    curr_url =  BASE_HTML_URL.replace(PLACEHOLDER_URL, curr_uuid)
    print('[{}]: Uploading UUID: {} | URL: {}'.format(row_name, curr_uuid, curr_url))
    html_bytes_io = io.BytesIO(generated_html.encode('utf-8'))
    upload_html_to_s3(html_bytes_io, BUCKET_NAME, curr_uuid + '.html')
    return curr_uuid, curr_url

In [219]:
missing_html_tags = []

In [220]:
def check_html(row):
    html = row['generated_html']
    if not'<!doctype html>' in html.strip().lower() or not '</html>' in html.strip().lower():
        missing_html_tags.append((row['method_name'], row['RICO GUI']))

# 1. Load Datasets

In [221]:
abs_path = '../data/generated/00_dataset_test/data/'

In [222]:
all_datasets = [
    # 1. Load all baseline results
    (1, '01_baselines_zs_instruction', abs_path+'01_baselines/zs_instruction.csv'),
    (2, '01_baselines_zs_cot', abs_path+'01_baselines/zs_cot.csv'),
    (3, '01_baselines_zs_cot_content', abs_path+'01_baselines/zs_cot_content.csv'),
    # 2. Load all prompt decompositon results
    (4, '02_prompt_decomposition_pd_zs', abs_path+'02_prompt_decomposition/pd_zs.csv'),
    (5, '02_prompt_decomposition_pd_zs_cot', abs_path+'02_prompt_decomposition/pd_zs_cot.csv'),
    (6, '02_prompt_decomposition_pd_zs_content', abs_path+'02_prompt_decomposition/pd_zs_content.csv'),
    # 3. Load all the RAG results
    (7, '03_rag_zs_k_1_direct', abs_path+'03_rag/rag_zs_k_1_direct.csv'),
    (8, '03_rag_zs_k_3_direct', abs_path+'03_rag/rag_zs_k_3_direct.csv'),
    (9, '03_rag_zs_k_5_direct', abs_path+'03_rag/rag_zs_k_5_direct.csv'),
    (10, '03_rag_zs_k_7_direct', abs_path+'03_rag/rag_zs_k_7_direct.csv'),
    # Rest of RAG results
    (11, '03_rag_zs_k_3_direct_content', abs_path+'03_rag/rag_zs_k_3_direct_content.csv'),
    (12, '03_rag_zs_k_3_direct_pd_zs', abs_path+'03_rag/rag_zs_k_3_direct_pd_zs.csv'),
    (13, '03_rag_zs_k_3_feat_layout', abs_path+'03_rag/rag_zs_k_3_feat_layout.csv'),
    # 4. Load all the self-critique results
    (14, '04_self_critique_base_zs_instruction_k_1', abs_path+'04_self_critique/base_zs_instruction/prototype_v1.csv'),
    (15, '04_self_critique_base_zs_instruction_k_2', abs_path+'04_self_critique/base_zs_instruction/prototype_v2.csv'),
    (16, '04_self_critique_base_zs_instruction_k_3', abs_path+'04_self_critique/base_zs_instruction/prototype_v3.csv'),
    (17, '04_self_critique_base_zs_instruction_k_4', abs_path+'04_self_critique/base_zs_instruction/prototype_v4.csv'),
    # Rest of self-critique results
    (18, '04_self_critique_base_zs_instruction_content_k_3', abs_path+'04_self_critique/base_zs_instruction_content/prototype_v3.csv'),
    (19, '04_self_critique_base_pd_zs_k_3', abs_path+'04_self_critique/base_pd_zs/prototype_v3.csv'),
    (20, '04_self_critique_base_rag_zs_k_3_direct_k_3', abs_path+'04_self_critique/base_rag_zs_k_3_direct/prototype_v3.csv'),
]

In [223]:
columns = ['RICO GUI', 'description', 'method_name', 'method_id', 'uuid', 'url']
master_df = pd.DataFrame(columns=columns)

In [ ]:
for (idd, name, path) in all_datasets:
    df_all = pd.read_csv(path)
    df_simplified = df_all[['RICO GUI', 'description', 'generated_html']].copy()
    df_simplified['method_name'] = name
    df_simplified['method_id'] = idd
    df_simplified.apply(check_html, axis=1)
    df_simplified[['uuid', 'url']] = df_simplified.apply(lambda row: pd.Series(generate_link_and_upload(row['generated_html'], row.name)), axis=1)
    df_simplified = df_simplified.drop(columns=['generated_html'])
    master_df = pd.concat([master_df, df_simplified], ignore_index=True)

In [226]:
master_df = master_df.sort_values(by=['RICO GUI', 'method_id'])
master_df['key'] = range(1, len(master_df) + 1)
master_df['key'] = master_df['key'].apply(lambda x: f"{x:04}")

In [227]:
master_df.to_csv('../data/evaluation_datasets/master_evaluation_table.csv', index=False)